## Inputs

In [143]:
table_name= "S_006_rollup_ap_drops"
table_name= table_name.lower()

In [144]:
text= """[Client Id] = [S_001_distinct_rollup_ap.Client Id]
[Rollup AP Month Name] = [S_001_distinct_rollup_ap.Rollup AP Month Name]
[Rollup AP Month First Date] = [S_001_distinct_rollup_ap.Rollup AP Month First Date]
[Rollup AP Month Last Date] = [S_001_distinct_rollup_ap.Rollup AP Month Last Date]
[Academic Drops] = [Drops.Enrollment Id]"""

## Process

In [145]:
import re
sp_text= list(map(lambda x: x.split('='), text.split('\n')))

In [146]:
#Variable label names

def clean_label(word):
    word= word.lower() \
        .strip() \
        .replace(' ', '_') \
        .replace('[', '') \
        .replace(']', '')
    return word

def is_nvarchar(word):
    if 'id' in word: return '[NVARCHAR] (250) NULL'
    else: return '#{Write var type}'

list_label_names= list(map(lambda x: clean_label(x[0]), sp_text))
list_labels= list(map(lambda x: 
                '['
                + x
                + '] '
                + is_nvarchar(x)
                , list_label_names))


In [147]:
#Table names

def clean_name(word):
    word= word.strip() \
        .replace('[', '') \
        .replace(']', '')

    return word

list_table_names= list(map(lambda x: clean_name(x[1].split('.')[0]), sp_text))

In [148]:
#Column names

list_column_names= list(map(lambda x: clean_name(x[1].split('.')[1]), sp_text))

## Desire Results

In [149]:
# Create table sypnase
print(
"""IF OBJECT_ID('acuity.{0}') IS NOT NULL
    DROP  TABLE [acuity].[{0}];
GO

CREATE TABLE [acuity].[{0}]
( 
        {1}
)
WITH
(
    DISTRIBUTION = ROUND_ROBIN,
	HEAP
)
GO
""".format(table_name,
        '\n\t,'.join(list_labels)))

IF OBJECT_ID('acuity.s_006_rollup_ap_drops') IS NOT NULL
    DROP  TABLE [acuity].[s_006_rollup_ap_drops];
GO

CREATE TABLE [acuity].[s_006_rollup_ap_drops]
( 
        [client_id] [NVARCHAR] (250) NULL
	,[rollup_ap_month_name] #{Write var type}
	,[rollup_ap_month_first_date] #{Write var type}
	,[rollup_ap_month_last_date] #{Write var type}
	,[academic_drops] #{Write var type}
)
WITH
(
    DISTRIBUTION = ROUND_ROBIN,
	HEAP
)
GO



In [150]:
#Create Stored Procedure
list_query_var= list(map(lambda x,y,z: 
                '['
                + x
                + '].['
                + y
                + '] AS '
                + z
                
                ,list_table_names,
                list_column_names,
                list_label_names))

print(
"""IF OBJECT_ID ( '[acuity].[sp_{0}]' ) IS NOT NULL   
    DROP PROCEDURE [acuity].[sp_{0}];  
GO  

CREATE PROC [acuity].[sp_{0}] AS BEGIN
    BEGIN TRANSACTION

        DECLARE @ErrorMessage NVARCHAR(4000);  
        DECLARE @ErrorSeverity INT;  
        DECLARE @ErrorState INT; 
        DECLARE @initial_time DATETIME;
        DECLARE @final_time DATETIME;
        DECLARE @time_diff_seconds FLOAT;
        DECLARE @table_name NVARCHAR(4000); 
        DECLARE @validate_query_result INT;

        BEGIN TRY  


            SET @initial_time  = CURRENT_TIMESTAMP;
		    SET @table_name = '{0}';

            
            DELETE FROM [acuity].[{0}]; 

            INSERT INTO [acuity].[{0}]
            SELECT 
                {1}

			
            SET @validate_query_result =  (SELECT count(*) FROM [acuity].[{0}]);

			IF @validate_query_result = 0
                BEGIN
                RAISERROR ('There are no rows in the source table', -- Message text.
                            16, -- Severity,
                            1 -- State,
                            );
                END ;

            SET @final_time  = CURRENT_TIMESTAMP;
            SET @time_diff_seconds = DATEDIFF(microsecond,@initial_time,@final_time) /1000000.0 ;

            INSERT INTO acuity.execution_history
                values(@table_name,@validate_query_result,'Table has been updated successfully',@time_diff_seconds , 'true', @final_time)


            COMMIT; 
            PRINT 'Table has been updated successfully'
        END TRY  
        BEGIN CATCH  
            ROLLBACK;
            
            SELECT   
                @ErrorMessage = ERROR_MESSAGE(),  
                @ErrorSeverity = ERROR_SEVERITY(),  
                @ErrorState = ERROR_STATE();  
        
            RAISERROR (@ErrorMessage, 
                    @ErrorSeverity, 
                    @ErrorState 
                    ); 

            SET @final_time  = CURRENT_TIMESTAMP;
            SET @time_diff_seconds = DATEDIFF(microsecond,@initial_time,@final_time) /1000000.0 ;
            
            INSERT INTO acuity.execution_history
            values(@table_name,@validate_query_result,@ErrorMessage,@time_diff_seconds , 'false', @final_time)

        END CATCH
END
GO
""".format(table_name,
        '\n\t\t\t\t,'.join(list_query_var)))

IF OBJECT_ID ( '[acuity].[sp_s_006_rollup_ap_drops]' ) IS NOT NULL   
    DROP PROCEDURE [acuity].[sp_s_006_rollup_ap_drops];  
GO  

CREATE PROC [acuity].[sp_s_006_rollup_ap_drops] AS BEGIN
    BEGIN TRANSACTION

        DECLARE @ErrorMessage NVARCHAR(4000);  
        DECLARE @ErrorSeverity INT;  
        DECLARE @ErrorState INT; 
        DECLARE @initial_time DATETIME;
        DECLARE @final_time DATETIME;
        DECLARE @time_diff_seconds FLOAT;
        DECLARE @table_name NVARCHAR(4000); 
        DECLARE @validate_query_result INT;

        BEGIN TRY  


            SET @initial_time  = CURRENT_TIMESTAMP;
		    SET @table_name = 's_006_rollup_ap_drops';

            
            DELETE FROM [acuity].[s_006_rollup_ap_drops]; 

            INSERT INTO [acuity].[s_006_rollup_ap_drops]
            SELECT 
                [S_001_distinct_rollup_ap].[Client Id] AS client_id
				,[S_001_distinct_rollup_ap].[Rollup AP Month Name] AS rollup_ap_month_name
				,[S_001_distinct_rollup_ap].[Ro